# 01 Data Exploration

This notebook profiles the three FFIEC source files, confirms join keys, and builds a data dictionary for the Similar Banks pipeline.

In [ ]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path('..').resolve().parent
land_path = DATA_DIR / 'Land___Clean_FFIEC_Data.csv'
loc_path = DATA_DIR / 'Locations__Bank_Suite_.csv'
inst_path = DATA_DIR / 'Institutions__Bank_Suite_.csv'

land = pd.read_csv(land_path, low_memory=False)
loc = pd.read_csv(loc_path, low_memory=False)
inst = pd.read_csv(inst_path, low_memory=False)

print('land shape', land.shape)
print('locations shape', loc.shape)
print('institutions shape', inst.shape)

In [ ]:
def profile(df, name):
    out = []
    n = len(df)
    for c in df.columns:
        s = df[c]
        out.append({
            'dataset': name,
            'column_name': c,
            'dtype': str(s.dtype),
            'row_count': n,
            'non_null_count': int(s.notna().sum()),
            'non_null_pct': round(float(s.notna().mean() * 100), 4),
            'n_unique': int(s.nunique(dropna=True))
        })
    return pd.DataFrame(out)

dictionary_df = pd.concat([
    profile(land, 'land'),
    profile(loc, 'locations'),
    profile(inst, 'institutions')
], ignore_index=True)
dictionary_df.head()

In [ ]:
land['Reporting_Period_End_Date'] = pd.to_datetime(land['Reporting_Period_End_Date'], errors='coerce')
latest_land_date = land['Reporting_Period_End_Date'].max()
land_latest = land.loc[land['Reporting_Period_End_Date'] == latest_land_date].copy()

inst['REPDTE'] = pd.to_datetime(inst['REPDTE'], errors='coerce')
inst_latest = (
    inst.sort_values(['FED_RSSD', 'REPDTE'])
    .groupby('FED_RSSD', as_index=False)
    .tail(1)
)

land_latest['IDRSSD'] = pd.to_numeric(land_latest['IDRSSD'], errors='coerce').astype('Int64')
inst_latest['FED_RSSD'] = pd.to_numeric(inst_latest['FED_RSSD'], errors='coerce').astype('Int64')
loc['CERT'] = pd.to_numeric(loc['CERT'], errors='coerce').astype('Int64')

joined = land_latest.merge(inst_latest[['FED_RSSD', 'CERT']], how='left', left_on='IDRSSD', right_on='FED_RSSD')
cert_set = set(loc['CERT'].dropna().astype('int64'))
coverage = joined['CERT'].dropna().astype('int64').isin(cert_set).mean()

print('Latest land date:', latest_land_date.date())
print('land latest banks:', land_latest['IDRSSD'].nunique())
print('join coverage IDRSSD->FED_RSSD:', joined['FED_RSSD'].notna().mean())
print('join coverage + CERT in locations:', coverage)

In [ ]:
out_path = Path('../output/data_dictionary_from_notebook.csv').resolve()
out_path.parent.mkdir(parents=True, exist_ok=True)
dictionary_df.to_csv(out_path, index=False)
print('Saved', out_path)